<a href="https://colab.research.google.com/github/prateekmanral011/hackathon_spg_slb/blob/main/Merged_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
import os

# ---------- INPUT / OUTPUT ----------
CONVERTED_CSV = "/content/A10_converted.csv"         # contains depth, poro, perm, gamma, facies...
FEATURES_CSV  = "/content/wells_split/A10_features.csv"  # contains MD and structural features (MD column name: "MD")
OUTPUT_CSV    = "/content/C7_merged_exact_nearestMD.csv"
# If you want to drop/blank feature columns when distance > threshold, set this to a value (meters).
# Otherwise set to None to keep all matches.
MAX_MATCH_DISTANCE = None   # e.g. 5.0 or None
# If you want to drop converted rows that have no petrophysical values after merge, set True
DROP_CONVERTED_WITHOUT_PETRO = False
# ------------------------------------

# 1) Read files
conv = pd.read_csv(CONVERTED_CSV).reset_index(drop=True)
feat = pd.read_csv(FEATURES_CSV).reset_index(drop=True)

print("Loaded: converted rows =", len(conv), "feature rows =", len(feat))

# 2) Ensure numeric depth columns exist and are numeric
# You said feature MD column is "MD"
FEATURE_MD_COL = "MD"
# Guess converted depth column (typical names)
possible = [c for c in conv.columns if c.lower() in ('depth','tvd','z','md','measureddepth')]
if len(possible) > 0:
    CONV_DEPTH_COL = possible[0]
else:
    # fallback to first numeric column
    numeric_cols = [c for c in conv.columns if pd.to_numeric(conv[c], errors='coerce').notna().sum() > 1]
    if not numeric_cols:
        raise ValueError("No numeric depth-like column found in converted CSV.")
    CONV_DEPTH_COL = numeric_cols[0]

print("Using converted depth column:", CONV_DEPTH_COL, "and feature MD column:", FEATURE_MD_COL)

conv[CONV_DEPTH_COL] = pd.to_numeric(conv[CONV_DEPTH_COL], errors='coerce')
feat[FEATURE_MD_COL] = pd.to_numeric(feat[FEATURE_MD_COL], errors='coerce')

conv = conv.dropna(subset=[CONV_DEPTH_COL]).reset_index(drop=True)
feat = feat.dropna(subset=[FEATURE_MD_COL]).reset_index(drop=True)

# 3) Build KDTree on feature MDs (1D)
feat_mds = feat[FEATURE_MD_COL].values.astype(float).reshape(-1,1)
tree = cKDTree(feat_mds)

# 4) Query nearest MD for every converted depth
conv_depths = conv[CONV_DEPTH_COL].values.astype(float).reshape(-1,1)
dists, idxs = tree.query(conv_depths, k=1)
dists = dists.flatten()
idxs = idxs.flatten()

# 5) Pull matched feature rows
matched_feat = feat.iloc[idxs].reset_index(drop=True).copy()

# 6) Prepare matched feature columns: rename to avoid collisions
# Keep matched MD as 'matched_MD' (original MD column will be copied into this)
matched_feat = matched_feat.rename(columns={FEATURE_MD_COL: 'matched_MD'})

# rename all other feature columns with suffix "_feat"
feat_cols = [c for c in matched_feat.columns if c != 'matched_MD']
rename_map = {c: f"{c}_feat" for c in feat_cols}
matched_feat = matched_feat.rename(columns=rename_map)

# 7) Compose merged DF (one row per converted depth)
merged = pd.concat([conv.reset_index(drop=True), matched_feat.reset_index(drop=True)], axis=1)
merged['md_distance_m'] = dists
merged['_matched_feat_index'] = idxs

# 8) Optionally blank feature columns for matches beyond MAX_MATCH_DISTANCE
if MAX_MATCH_DISTANCE is not None:
    mask_ok = merged['md_distance_m'].abs() <= float(MAX_MATCH_DISTANCE)
    if (~mask_ok).any():
        print(f"{(~mask_ok).sum()} rows have md_distance_m > {MAX_MATCH_DISTANCE} and will have feature columns set to NaN.")
        featcols = [c for c in merged.columns if c.endswith('_feat') or c == 'matched_MD']
        merged.loc[~mask_ok, featcols] = np.nan
    else:
        print("All matches are within MAX_MATCH_DISTANCE.")

# 9) (Optional) Remove converted rows that do not have any petrophysical data after merge
if DROP_CONVERTED_WITHOUT_PETRO:
    # determine petrophysical columns by common names
    petros = [c for c in merged.columns if any(s in c.lower() for s in ('por','perm','phi','gamma','gr','facies'))]
    if len(petros) == 0:
        print("No petrophysical columns auto-detected; not dropping any rows.")
    else:
        mask_has = merged[petros].notna().any(axis=1)
        removed = (~mask_has).sum()
        merged = merged[mask_has].copy().reset_index(drop=True)
        print(f"Dropped {removed} converted rows that had no petrophysical values.")

# 10) Save
merged.to_csv(OUTPUT_CSV, index=False)
print("Merged saved to:", OUTPUT_CSV)
print("Merged shape:", merged.shape)
print("md_distance_m stats:", merged['md_distance_m'].min(), merged['md_distance_m'].median(), merged['md_distance_m'].max())

# 11) Quick sample view
display_cols = [CONV_DEPTH_COL, 'matched_MD', 'md_distance_m'] + list(conv.columns[:8])
print("Sample merged rows (first 40):")
display(merged[[CONV_DEPTH_COL, 'matched_MD', 'md_distance_m'] + list(merged.columns[3:10])].head(40))


Loaded: converted rows = 1835 feature rows = 6011
Using converted depth column: DEPT and feature MD column: MD
Merged saved to: /content/C7_merged_exact_nearestMD.csv
Merged shape: (1835, 27)
md_distance_m stats: 7.999999979801942e-06 0.03809100000012222 0.5762089999998352
Sample merged rows (first 40):


,DEPT,matched_MD,md_distance_m,POROSITY,FLUVIALFACIES,NETGROSS,WellID_feat,X_feat,Y_feat,TVD_feat
0,1499.879,1499.878992,0.000008,NaN,NaN,0.0,A10,456979.0637,6782712.412,1499.878992
1,1500.129,1500.183692,0.054692,NaN,NaN,0.0,A10,456979.2657,6782712.377,1500.104433
2,1500.629,1500.640892,0.011892,NaN,NaN,0.0,A10,456979.5687,6782712.324,1500.442675
3,1501.129,1501.097992,0.031008,0.270646,0.0,0.0,A10,456979.8718,6782712.272,1500.780807
4,1501.629,1501.555192,0.073808,0.267428,0.0,0.0,A10,456980.1750,6782712.219,1501.118977
5,1502.129,1502.164892,0.035892,0.256076,0.0,0.0,A10,456980.5793,6782712.149,1501.569886
6,1502.629,1502.622092,0.006908,0.242126,0.0,0.0,A10,456980.8826,6782712.096,1501.907967
7,1503.129,1503.079192,0.049808,0.238589,0.0,0.0,A10,456981.1858,6782712.044,1502.245936
8,1503.629,1503.688792,0.059792,0.238377,0.0,0.0,A10,456981.5903,6782711.974,1502.696604
9,1504.129,1504.146092,0.017092,0.240893,0.0,0.0,A10,456981.8938,6782711.921,1503.034636
